EPL: league 39

In [5]:
import requests
import json
import os

from dotenv import load_dotenv
load_dotenv()


True

# Get Scores from EPL on date

In [6]:
date = '2022-08-14'

In [7]:
# get all the ids of a match for a url 
url = "https://v3.football.api-sports.io/fixtures"

payload={'league':39, 'season':2022, 'date': date}

headers = {
    'x-rapidapi-key': os.environ.get("RAPIDAPI_KEY"),
  'x-rapidapi-host': 'v3.football.api-sports.io'
}

fixtures_response = json.loads(requests.request("GET", url, headers=headers, params=payload).text)


In [8]:
score_match_dict = {}
for match in fixtures_response['response']:
    # get the fixture id
    fixture = match['fixture']['id']
    
     # get the team names
    home_team = match['teams']['home']['name']
    away_team = match['teams']['away']['name']

    # get the score
    home_score = match['score']['fulltime']['home']
    away_score = match['score']['fulltime']['away']

    # who won?
    if home_score > away_score:
        winner = 'Home'
    elif home_score < away_score:
        winner = 'Away'
    else:
        winner = 'Draw'

    score_match_dict[fixture]={'home_team':home_team, 'away_team':away_team, 'home_score':home_score, 'away_score':away_score, 'winner':winner}


# Get the "Odds of Winning" for each game on the date

In [9]:

url = "https://v3.football.api-sports.io/odds"

payload={'league':39, 'season':2022, 'date': date}

headers = {
    'x-rapidapi-key': os.environ.get("RAPIDAPI_KEY"),
  'x-rapidapi-host': 'v3.football.api-sports.io'
}

odds_response = json.loads(requests.request("GET", url, headers=headers, params=payload).text)


In [10]:
odds_match_dict = {}
for match in odds_response['response']:

    # check to make sure 0 == match winner
    if match['bookmakers'][0]['bets'][0]['name'] != 'Match Winner':
        print('oh no! first id is not match winner')

    odd_dict = {}
    for value in match['bookmakers'][0]['bets'][0]['values']:
        odd_dict[value['odd']] = value['value']

    pred_win = odd_dict[min(odd_dict.keys())]

    
    
    odds_match_dict[match['fixture']['id']] = pred_win

# Print Outputs

In [11]:
# get lists of correct and incorrect predictions
correct_ids = []
upset_ids = []
for match_id in odds_match_dict.keys():
    if odds_match_dict[match_id]!=score_match_dict[match_id]['winner']:
       upset_ids.append(match_id) 
    else:
        correct_ids.append(match_id)

print('#EPL Scores {date}'.format(date=date))
print("SURPRISES:")
for id in upset_ids:
    score_match_dict[id]
    print(str("{home_team}: {home_score} - {away_team}: {away_score}").format(
        home_team=score_match_dict[id]['home_team'], 
        home_score=score_match_dict[id]['home_score'], 
        away_team=score_match_dict[id]['away_team'], 
        away_score=score_match_dict[id]['away_score']))

print("\nEXPECTED:")
for id in correct_ids:
    score_match_dict[id]
    print(str("{home_team}: {home_score} - {away_team}: {away_score}").format(
        home_team=score_match_dict[id]['home_team'], 
        home_score=score_match_dict[id]['home_score'], 
        away_team=score_match_dict[id]['away_team'], 
        away_score=score_match_dict[id]['away_score']))

#EPL Scores 2022-08-14
UPSETS:
Chelsea: 2 - Tottenham: 2
Nottingham Forest: 1 - West Ham: 0

EXPECTED:
